# Downloading the HCP 7T retinotopy dataset

The data is available at https://osf.io/83wkp/.

In this notebook, we'll download the prfresults.mat, convert it to ciftis, convert those to subcortical niftis. Although the depository also provides a direct download of subcortical niftis, these are not in native MNI space (but in a smaller bounded box). 

The HCP pRF data can be downloaded directly from the OSF website using python. However, the timeseries data (needed to get a cifti model) has to be downloaded by hand from the HCP website: https://db.humanconnectome.org. You'll need to create a free account.

In addition, you'll need to have the HCP workbench software installed on your computer to do the cifti-nifti conversions: https://www.humanconnectome.org/software/connectome-workbench

In [46]:
import urllib
import cifti
import numpy as np
import os
import nibabel as nb
import h5py

### define directories

In [40]:
# change these directories accordingly 
data_dir = '/home/shared/2018/visual/hcp_cerebellum/'
all_sub_dir = os.path.join(data_dir,'all_subjects')
if not os.path.isdir(all_sub_dir): os.mkdir(all_sub_dir)
mat_fn = os.path.join(data_dir,'prfresults.mat')
prf_mat_url = 'https://osf.io/yus6t/download'

# for the timeseries
ts_dir =  os.path.join(data_dir,'timeseries')
if not os.path.isdir(ts_dir): os.mkdir(ts_dir)
ts_gii = os.path.join(ts_dir,'tfMRI_RETBAR1_7T_AP_Atlas_MSMAll_hp2000_clean.dtseries.nii')

### download the pRF results

In [34]:
if not os.path.isfile(mat_fn):
    print('Downloading HCP 7T pRF results...')
    urllib.urlretrieve(prf_mat_url, mat_fn)  
    print('downloading done!')
else:
    print('prfmatfile already downloaded')

prfmatfile already downloaded


### get the cifti brain model

In [49]:
# get the cifti brain model
bm = cifti.read(ts_gii)[1][1] 

# the workbench command to convert subcortical cifti to nifti
wbc = '/home/vanes/Downloads/workbench/bin_rh_linux64/wb_command -cifti-separate {cii} \
COLUMN -volume-all {cii_n}_data_sub.nii.gz'
# -metric CORTEX_LEFT {cii_n}_L.gii \
#-metric CORTEX_RIGHT {cii_n}_R.gii &"""

### load the prf results

In [39]:
# # read prf results for all subjects:
with h5py.File(mat_fn, 'r') as mat:
    allresults = mat['/allresults'].value
    
# # for details on this file see: https://osf.io/bw9ec/wiki/home/
# # this is a huge numpy array now, with dimensions:
# # 0: run half
# #        0 = all data
# #        1 = first half of runs
# #        2 = second half of runs
# # 1: subjects (len = 184; 181 individual subjects and 3 average subjects)
# # 2: prf param (len = 6)
# #        0 = polar angle (in degrees from 0-360)
# #        1 = ecc
# #        2 = gain
# #        3 = mean_signal
# #        4 = r2
# #        5 = rfsize
# # 3: grayordinates (len = 91282)

### now create ciftis and niftis of all subjects

In [52]:
# For all subjects (this takes some time)
for sj in range(184):
    
    print 'now converting results.mat to cifti for subject %d'%sj
    these_results = copy.copy(allresults[0,sj]) # 0 refers to the first fit (so on all data)
    
    # save results
    cii = os.path.join(all_sub_dir,'prfresults_subject_%d.dscalar.nii'%sj)
    cifti.write(cii, these_results, (cifti.Scalar.from_names(['ang', 'ecc','gain', 'meanvol', 'R2', 'rfsize']), bm))

    # and convert the created cifti file to gii files
    print 'now converting cifti to subcortical nifti for subject %d'%sj
    wbc_c = wbc.format(cii=cii, cii_n=cii[:-4])
    os.system(wbc_c)    
    
    # now remove the cifti file (we don't need that anymore, just takes up space)
    os.remove(cii)

print('done!')

now converting results.mat to cifti for subject 0
now converting cifti to subcortical nifti for subject 0
now converting results.mat to cifti for subject 1
now converting cifti to subcortical nifti for subject 1
now converting results.mat to cifti for subject 2
now converting cifti to subcortical nifti for subject 2
now converting results.mat to cifti for subject 3
now converting cifti to subcortical nifti for subject 3
now converting results.mat to cifti for subject 4
now converting cifti to subcortical nifti for subject 4
now converting results.mat to cifti for subject 5
now converting cifti to subcortical nifti for subject 5
now converting results.mat to cifti for subject 6
now converting cifti to subcortical nifti for subject 6
now converting results.mat to cifti for subject 7
now converting cifti to subcortical nifti for subject 7
now converting results.mat to cifti for subject 8
now converting cifti to subcortical nifti for subject 8
now converting results.mat to cifti for subjec

### convert timeseries gii

In [53]:
# convert timeseries gifti to nifti:
fns = ['tfMRI_RETBAR1_7T_AP_Atlas_MSMAll_hp2000_clean.dtseries.nii',
       'tfMRI_RETBAR2_7T_PA_Atlas_MSMAll_hp2000_clean.dtseries.nii']

for fn in fns:
    cii = os.path.join(ts_dir,fn)

    # and convert the created cifti file to gii files
    wbc_c = wbc.format(cii=cii, cii_n=cii[:-4])
    os.system(wbc_c)
    

### and average the two runs for increased SNR

In [54]:
# average two runs
fns = ['tfMRI_RETBAR1_7T_AP_Atlas_MSMAll_hp2000_clean.dtseries_data_sub.nii.gz',
      'tfMRI_RETBAR2_7T_PA_Atlas_MSMAll_hp2000_clean.dtseries_data_sub.nii.gz']

data = []
for fn in fns:
    
    nii_fn = os.path.join(ts_dir,fn)
    nii_img = nb.load(nii_fn)

    data.append(nii_img.get_data())

mean_data = np.mean(data,axis=0)
new_img = nb.Nifti1Image(mean_data,header=nii_img.header,affine=nii_img.affine)
nb.save(new_img,os.path.join(out_dir,'timeseries','avg_prf_timeseries.nii.gz'))
